In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn

In [3]:
# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')
details = pd.read_csv('data/data_description.csv', encoding='latin1')

# Rename columns to english for ease of use
data.rename(columns=dict(zip(details['Feature (nl)'], details['Feature (en)'])), inplace=True)

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Here we will look at different features and their format to discover potential validity problems in the data

### First, we will look at the features related to the contacts with the municipality

#### Since we know all of those attributes represent numbers of appointments, signaling requirement with administrative help, we expect this attribute to have little impact on the risk calculation, and thus will use metamorphic testing on this relationship

In [13]:
# Define metamorphic transformations
def perturbation(df, cols):
    df_perturbed = df.copy()
    for col in cols:
        df_perturbed[col] = df[col] + np.random.normal(0, df[col].std(), size=df[col].shape)
    return df_perturbed

def permutation(df, cols):
    df_permuted = df.copy()
    for col in cols:
        df_permuted[col] = np.random.permutation(df[col])   
    return df_permuted

In [22]:
# Commented out because this just visualises desired fields
if False: 
  plt.clf()
  apps = [col for col in data.columns if 'appointment' in col and 'number' not in col and 'combined' not in col]
  # Combine Appointment values in a new feature
  data["appointment_combined"] = data[apps].sum(axis=1)
  data.boxplot(column=['appointment_combined'])
  plt.show()
  data.drop(columns='appointment_combined', inplace=True)

  # Address Exploration

  plt.clf()
  adds = [col for col in data.columns if 'address' in col]
  for row in details.itertuples():
      if row[3] in adds:
          print(f"{row[3]} :::: {row[6]}")
  data.hist(column=adds, figsize=(25, 15))
  plt.show()

  plt.clf()
  contacts = [col for col in data.columns if 'contacts' in col]
  for row in details.itertuples():
      if row[3] in contacts:
          print(f"{row[3]} :::: {row[6]}")
  data.hist(column=contacts, figsize=(25, 15))
  plt.show()

  plt.clf()
  obstacles = [col for col in data.columns if 'obstacle' in col]
  for row in details.itertuples():
      if row[3] in obstacles:
          print(f"{row[3]} :::: {row[6]}")
  data.hist(column=obstacles, figsize=(25, 15))
  plt.show()

  plt.clf()
  avail = [col for col in data.columns if 'availability' in col]
  for row in details.itertuples():
      if row[3] in avail:
          print(f"{row[3]} :::: {row[6]}")
  data.hist(column=avail, figsize=(25, 15))
  plt.show()



# Set up metamorphic relationships
metamorphic_relations=[
    {'name': 'appointments', 'target': lambda df: [col for col in df.columns if 'appointment' in col],
      'transformations': [{'name': 'permutation', 'function': permutation}, {'name': 'perturbation', 'function': perturbation}]},
    {'name': 'address_numerical', 'target': lambda df: [col for col in df.columns if 'address_number' in col or 'address_days' in col],
      'transformations': [{'name': 'permutation', 'function': permutation}, {'name': 'perturbation', 'function': perturbation}]},
    {'name': 'address_binary', 'target': lambda df: [col for col in df.columns if 'address_latest' in col or 'address_unique' in col],
      'transformations': [{'name': 'permutation', 'function': permutation}]},
]




In [ ]:
# Define equivalent partitions
partition_tests = [
    [
    # Related to obstacles
    {"name": "fin_0", "condition": lambda df: df['obstacle_financial_problems'] < 0.5},
    {"name": "fin_1", "condition": lambda df: df['obstacle_financial_problems'] > 0.5}
    ],[
    {"name": "psyc_0", "condition": lambda df: df['obstacle_hist_psychological_problems'] < 0.5 and df['obstacle_psychological_problems'] < 0.5},
    {"name": "psyc_1", "condition": lambda df: df['obstacle_hist_psychological_problems'] > 0.5 and df['obstacle_psychological_problems'] > 0.5}
    ],[
    {"name": "lan_0", "condition": lambda df: df['obstacle_hist_language'] < 0.5},
    {"name": "lan_1", "condition": lambda df: df['obstacle_hist_language'] > 0.5}
    ],[
    {"name": "addict_0", "condition": lambda df: df['obstacle_hist_addiction_problems'] < 0.5},
    {"name": "addict_1", "condition": lambda df: df['obstacle_hist_addiction_problems'] > 0.5}
    ],
    # Related to availability deviations
    [
    {"name": "social_0", "condition": lambda df: df['availability_recent_deviating_due_to_social_social_situation'] < 0.5},
    {"name": "social_1", "condition": lambda df: df['availability_recent_deviating_due_to_social_social_situation'] > 0.5}
    ],[
    {"name": "medical_0", "condition": lambda df: df['availability_recent_deviating_due_to_medical_circumstances'] < 0.5},
    {"name": "medical_1", "condition": lambda df: df['availability_recent_deviating_due_to_medical_circumstances'] > 0.5}
    ],

    
]

In [ ]:
# Apply metamorphic testing

for metamorphic in metamorphic_relations:
    target_columns = metamorphic['target'](data_sample)
    print(f"Test: {metamorphic["name"]}")


    for transformation in metamorphic['transformations']:
        
        print(f"{transformation['name']} test")
        acc_list = []
        for run in range(10):
 
            data_sample = X_test.sample(n=500)

            tranf_data = transformation['function'](data_sample, target_columns)
            tranf_labels = y_test.loc[tranf_data.index]

            if not tranf_data.empty:
            # Predictions using the model
                predictions = model.predict(tranf_data)

                # Calculate accuracy for this partition
                accuracy = accuracy_score(tranf_labels, predictions)
                acc_list.append(accuracy)

                # Print transformation details
                print(f"Run: {run}")
                print(f"Accuracy: {accuracy:.2f}")
                print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")
                print()

        print("Final Results")
        print(f"Average accuracy: {np.mean(acc_list)}")

In [ ]:
# Apply equivalent partitioning
# Needs some work with multiple train/test splits
for partitions in partition_tests:
    for partition in partitions:
        partition_data = data_sample[partition["condition"](data_sample)]
        partition_indices = partition_data.index  # Get the indices of the partition
        partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

        if not partition_data.empty:
            # Predictions using the model
            predictions = model.predict(partition_data)

            # Calculate accuracy for this partition
            accuracy = accuracy_score(partition_labels, predictions)

            # Print partition details
            print(f"Partition: {partition['name']}")
            print(f"Number of data points: {len(partition_data)}")
            print(f"Accuracy: {accuracy:.2f}")
            print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

In [3]:
# Select data based on variance (not the final version yet, for now just for testing)
selector = VarianceThreshold()

In [4]:
# Define a gradient boosting classifier
classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

In [5]:
# Create a pipeline object with our selector and classifier
# NOTE: You can create custom pipeline objects but they must be registered to onnx or it will not recognise them
# Because of this we recommend using the onnx known objects as defined in the documentation
pipeline = Pipeline(steps=[('feature selection', selector), ('classification', classifier)])

In [ ]:
# Let's train a simple model
pipeline.fit(X_train, y_train)

# Let's evaluate the model
y_pred = pipeline.predict(X_test)
original_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the original model: ', original_accuracy)

Accuracy of the original model:  0.9456040480708412


: 

In [ ]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Let's check the accuracy of the converted model
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

In [ ]:
# Let's save the model
onnx.save(onnx_model, "model/gboost.onnx")

# Let's load the model
new_session = rt.InferenceSession("model/gboost.onnx")

# Let's predict the target
y_pred_onnx2 =  new_session.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)
